## Load necessary modules

In [7]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# use this environment flag to change which GPU to use
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
keras.backend.tensorflow_backend.set_session(get_session())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load RetinaNet model

In [5]:
# adjust this to point to your downloaded/trained model
# models can be downloaded here: https://github.com/fizyr/keras-retinanet/releases
model_path = os.path.join('..', 'snapshots', 'single25.h5')
print(model_path)

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')

# if the model is not converted to an inference model, use the line below
# see: https://github.com/fizyr/keras-retinanet#converting-a-training-model-to-inference-model
#model = models.load_model(model_path, backbone_name='resnet50', convert_model=True)

#print(model.summary())

# load label to names mapping for visualization purposes
labels_to_names = {0: '111'}

../snapshots/single25.h5


/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:270: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [3]:
ls ../snapshots/

Parameter format not correct - "snapshots".


## Run detection on example

In [6]:

with open('testimages.txt') as f:
    
    for line in f:
        try:
            # load image
            image = read_image_bgr('test_images/' + line[:-1])

            # copy to draw on
            draw = image.copy()
            draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

            # preprocess image for network
            image = preprocess_image(image)
            image, scale = resize_image(image)

            # process image
            start = time.time()
            boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
            print("processing time: ", time.time() - start)

            outfile = 'output/' +line[:-5] + '.txt'
            fout = open(outfile, mode = 'w+')

            # correct for image scale
            boxes /= scale

            # visualize detections
            for box, score, label in zip(boxes[0], scores[0], labels[0]):
                # scores are sorted so we can break
                if score < 0.01:
                    break

                color = label_color(label)

                b = box.astype(int)
                fout.write(f'{b[1]} {b[0]} {b[3]} {b[2]}\n')
                #print(b , ' ', score, ' ' , label)
                draw_box(draw, b, color=color)

                caption = "{} {:.3f}".format(labels_to_names[label], score)
                draw_caption(draw, b, caption)

            fout.close()
            plt.figure(figsize=(15, 15))
            plt.axis('off')
            plt.imshow(draw)
            plt.show()
        
        except:
            print(line[:-1])


processing time:  3.431411027908325
1ROI_100kx_4100CL_foil1.jpg
processing time:  2.6552000045776367
200kV_500kx_p2nm_8cmCL_grain1_0004_n.jpg
processing time:  2.6706650257110596
200kV_500kx_p2nm_8cmCL_grain1_0036.jpg
processing time:  2.6541569232940674
200kV_500kx_p2nm_8cmCL_grain1_0056 - Copy.jpg
processing time:  2.6502957344055176
200kV_500kx_p2nm_8cmCL_grain1_0090 - Copy.jpg
processing time:  2.6629319190979004
200kV_500kx_p2nm_8cmCL_grain1_0135 - Copy.jpg
processing time:  2.658515214920044
200kV_500kx_p2nm_8cmCL_grain2_0008.jpg
processing time:  2.7246150970458984
200kV_500kx_p2nm_8cmCL_grain2_0034.jpg
processing time:  2.6781020164489746
200kV_500kx_p2nm_8cmCL_grain2_0036.jpg
processing time:  2.6763341426849365
0501_300kx_1nm_clhaadf3_0028.jpg
processing time:  2.6501100063323975
2501_300kx_1nm_clhaadf3_0042.jpg
processing time:  2.6815249919891357
2501_300kx_1nm_clhaadf3_0052.jpg
processing time:  2.659557342529297
g1_backonzone_GBtowardmiddle_0001.jpg
processing time:  2.66